<a href="https://colab.research.google.com/github/konstantinstanmeyer/music-extractor-1/blob/main/find_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports

import os # validates if audio files already exist locally


In [ ]:
# file downloads

if !os.path.isfile('wish-you-were-here.m4a'):
  print("file downloads starting...")

In [ ]:
FFT_WINDOW_SECONDS = 0.25